In [112]:
import numpy as np
import pandas as pd
from os.path import join
from pdb import set_trace as st
from numpy import random
from scipy import stats
import sys
import pickle

In [114]:
def report_progress(progress, total, lbar_prefix = '', rbar_prefix=''):
    percent = round(progress / float(total) * 100, 2)
    buf = "{0}|{1}| {2}{3}/{4} {5}%".format(lbar_prefix, ('#' * round(percent)).ljust(100, '-'),
        rbar_prefix, progress, total, percent)
    sys.stdout.write(buf)
    sys.stdout.write('\r')
    sys.stdout.flush()

In [115]:
data_type = "0304"
file_path = f"./lsx_{data_type}.csv"
ages = [0, 19, 31, 51, np.inf]
stages = ["normal_bp", "elevated", "stage_1", "stage_2", "stage_3"]
iterations = 1000

In [116]:
def operate():
    # Load CSV
    df = pd.read_csv(file_path)
    
    # Start Empty Table for Track Result
    results = dict()
    
    for i in range(len(ages)-1):
        start_age, end_age = ages[i], ages[i+1] # [0, 19]
        
        print("\n=============================================")
        print(f"Age {start_age} ~ {end_age}")
        print("=============================================")
        
        df_age = df[df['age'] >= start_age]
        df_age = df_age[df_age['age'] < end_age]
        
        result = dict()
        
        for j in range(len(stages)-1):
            for k in range(j+1, len(stages)):
                
                print("\n---------------------------------------------")
                print(f"Sets {j} & {k}")
                print("---------------------------------------------")
        
                # Get name of the paired sets
                cls_1 = stages[j]
                cls_2 = stages[k]
                # Get set data
                set1 = df_age[df_age[cls_1] == True]
                set2 = df_age[df_age[cls_2] == True]
                # Get sample size
                N = min(set1.shape[0], set2.shape[0])
                # Iterations
                distances = list()
                for i in range(iterations):
                    # Two random indices, one for each set, sample size N
                    random_index_1 = random.choice(set1.shape[0], N)
                    random_index_2 = random.choice(set2.shape[0], N)
                    # Get the two samples, each [N, 7200]
                    sample_1 = set1.iloc[random_index_1, :]
                    sample_2 = set2.iloc[random_index_2, :]
                    # Get column-wise median
                    median_1 = np.median(sample_1, axis=0)
                    median_2 = np.median(sample_2, axis=0)
                    distance = max(median_1 - median_2)
                    distances.append(distance)
                    report_progress(i, iterations)
                distances = np.array(distances)
                mean, sigma = np.mean(distances), np.std(distances)
                conf_int = stats.norm.interval(0.95, loc=mean, scale=sigma) # / np.sqrt(len(distances)))
                # Check if 0 is in between
                if 0 >= conf_int[0] and 0 <= conf_int[1]:
                    result[(j, k)] = -1 # p < 0.05 two sided
                else:
                    t_value = (mean - 0) / (std / np.sqrt(len(distances))) # t-statistic for mean
                    pval = stats.t.sf(np.abs(t_value), len(distances) - 1) * 2  # two-sided pvalue = Prob(abs(t)>tt)
                    result[(j, k)] = pval
                    
        results[(start_age, end_age)] = result
        
    with open(f"./{data_type}_results.pkl", "wb") as file:
        pickle.dump(results, file)
    return results

In [117]:
distances = operate()


Age 0 ~ 19

---------------------------------------------
Sets 0 & 1
---------------------------------------------
|####################################################################################################| 999/1000 99.9%
---------------------------------------------
Sets 0 & 2
---------------------------------------------
|####################################################################################################| 999/1000 99.9%
---------------------------------------------
Sets 0 & 3
---------------------------------------------
|####################################################################################################| 999/1000 99.9%
---------------------------------------------
Sets 0 & 4
---------------------------------------------


/home/lishixuan001/installations/miniconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)


ZeroDivisionError: division by zero